In [9]:
import os
import sqlite3
import json
import pandas as pd

from openai import OpenAI

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

#### Initialize Environment and Client

In [4]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [10]:
conn = sqlite3.connect("products.db")

#### Use an LLM to Query a Database

In [5]:
prompt = """
You are a SQL assistant. Given the schema and the user's question, write a SQL query for SQLite.

Schema:
{schema}

User question:
{question}

Respond with the SQL only.
"""

In [7]:
schema = """
Table name: transactions
id (INTEGER)
product_id (INTEGER)
product_name (TEXT)
brand (TEXT)
category (TEXT)
color (TEXT)
action (TEXT)
qty_delta (INTEGER)
unit_price (REAL)
notes (TEXT)
ts (DATETIME)
"""
response = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[
        {
            "role": "user",
            "content": prompt.format(
                schema=schema,
                question="Which color of product has the highest total sales?"
            )
        }
    ],
)
sql_V1 = response.choices[0].message.content.strip()

In [8]:
print(sql_V1)

SELECT color
FROM (
  SELECT color, SUM(qty_delta * unit_price) AS total_sales
  FROM transactions
  WHERE qty_delta > 0
  GROUP BY color
)
ORDER BY total_sales DESC
LIMIT 1;


In [11]:
pd.read_sql_query(sql_V1, conn)

,color
0,white


#### Improving SQL Queries with Reflection

In [19]:
prompt_2 = """
You are a SQL reviewer and refiner.

User asked:
{question}

Original SQL:
{sql_query}

Table Schema:
{schema}

Step 1: Briefly evaluate if the SQL OUTPUT fully answers the user's question.
Step 2: If improvement is needed, provide a refined SQL query for SQLite.
If the original SQL is already correct, return it unchanged.

Return STRICT JSON with two fields:
{{
  "feedback": "<1-3 sentences explaining the gap or confirming correctness>",
  "refined_sql": "<final SQL to run>"
}}
"""

In [20]:
response_2 = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[
        {
            "role": "user",
            "content": prompt_2.format(
                schema=schema,
                question="Which color of product has the highest total sales?",
                sql_query=sql_V1
            )
        }
    ],
)
reflection = response_2.choices[0].message.content

In [21]:
print(reflection)

{
  "feedback": "The original query mostly computes sales per color correctly but the derived table needs an alias in SQLite and it would be helpful to return the sales amount and handle NULL prices/colors and possible ties. The refined query below fixes those issues and returns all color(s) that have the maximum total sales.",
  "refined_sql": "WITH sales_by_color AS (\n  SELECT color,\n         SUM(qty_delta * COALESCE(unit_price, 0.0)) AS total_sales\n  FROM transactions\n  WHERE qty_delta > 0\n    AND color IS NOT NULL\n  GROUP BY color\n)\nSELECT color, total_sales\nFROM sales_by_color\nWHERE total_sales = (SELECT MAX(total_sales) FROM sales_by_color);"
}


In [24]:
reflection_json = json.loads(reflection)
feedback = str(reflection_json.get("feedback", "")).strip()
refined_sql = str(reflection_json.get("refined_sql", sql_V1)).strip()

In [26]:
print(refined_sql)

WITH sales_by_color AS (
  SELECT color,
         SUM(qty_delta * COALESCE(unit_price, 0.0)) AS total_sales
  FROM transactions
  WHERE qty_delta > 0
    AND color IS NOT NULL
  GROUP BY color
)
SELECT color, total_sales
FROM sales_by_color
WHERE total_sales = (SELECT MAX(total_sales) FROM sales_by_color);


In [29]:
df_sql_V1 = pd.read_sql_query(sql_V1, conn)
df_sql_V2 = pd.read_sql_query(refined_sql, conn)

print("Initial SQL output:")
print(df_sql_V1)

print("\nRefined SQL output:")
print(df_sql_V2)

print(f"\nFeedback: {feedback}")

Initial SQL output:
   color
0  white

Refined SQL output:
   color  total_sales
0  white     68437.36

Feedback: The original query mostly computes sales per color correctly but the derived table needs an alias in SQLite and it would be helpful to return the sales amount and handle NULL prices/colors and possible ties. The refined query below fixes those issues and returns all color(s) that have the maximum total sales.


#### Final Approach: Refine an SQL Query with External Feedback

In [31]:
prompt_3 = """
You are a SQL reviewer and refiner.

User asked:
{question}

Original SQL:
{sql_query}

SQL Output:
{df_feedback}

Table Schema:
{schema}

Step 1: Briefly evaluate if the SQL output answers the user's question.
Step 2: If the SQL could be improved, provide a refined SQL query.
If the original SQL is already correct, return it unchanged.

Return a strict JSON object with two fields:
- "feedback": brief evaluation and suggestions
- "refined_sql": the final SQL to run
"""

In [33]:
response_3 = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[
        {
            "role": "user",
            "content": prompt_3.format(
                schema=schema,
                question="Which color of product has the highest total sales?",
                sql_query=sql_V1,
                df_feedback=df_sql_V1.to_markdown(index=False)
            )
        }
    ],
)
reflection_3 = response_3.choices[0].message.content

In [34]:
reflection_json_3 = json.loads(reflection_3)
feedback_2 = str(reflection_json_3.get("feedback", "")).strip()
sql_V2 = str(reflection_json_3.get("refined_sql", sql_V1)).strip()

In [35]:
print(sql_V2)

SELECT color, SUM(qty_delta * unit_price) AS total_sales
FROM transactions
WHERE qty_delta > 0
GROUP BY color
ORDER BY total_sales DESC
LIMIT 1;


,color,total_sales
0,white,68437.36


In [38]:
df_sql_V1 = pd.read_sql_query(sql_V1, conn)
df_sql_V2 =pd.read_sql_query(sql_V2, conn)

print("Initial SQL output:")
print(df_sql_V1)

print("\nRefined SQL output:")
print(df_sql_V2)

print(f"\nFeedback: {feedback_2}")

Initial SQL output:
   color
0  white

Refined SQL output:
   color  total_sales
0  white     68437.36

Feedback: The intended logic is correct (sum of qty_delta * unit_price per color, considering only positive qty_delta). However, the original SQL uses an unnecessary derived table without an alias and orders by total_sales from the outer query — which will fail in many SQL engines. It's simpler and more robust to aggregate and order in a single query. I also return total_sales along with the color for clarity.


In [40]:
conn.close()